## Deliverable 3. Create a Travel Itinerary Map.

In [1]:
#Enable widget to see figures in notebook
!jupyter nbextension enable --py --sys-prefix widgetsnbextension
!jupyter nbextension enable --py --sys-prefix gmaps

# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import gmaps.datasets

# imports the previous directory for config.py
import sys
sys.path.insert(1, '../')

# Import API key
from config import g_key

# Configure gmaps
gmaps.configure(api_key=g_key)

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: ok
Enabling notebook extension jupyter-gmaps/extension...
      - Validating: ok


In [2]:
# 1. Read the WeatherPy_vacation.csv into a DataFrame.
vacation_df = pd.read_csv("../Vacation_Search/WeatherPy_vacation.csv")
vacation_df.set_index('City')
vacation_df.drop(labels=['Unnamed: 0'],axis=1, inplace=True)
vacation_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Saint-Philippe,RE,77.00,few clouds,-21.36,55.77,"Chambres d'hôte ""La Trinité"""
1,Itarema,BR,84.70,clear sky,-2.92,-39.92,Windvillage
2,Takoradi,GH,79.09,clear sky,4.88,-1.76,Raybow International Hotel
3,Limon,CR,82.40,broken clouds,10.00,-83.03,Hotel Playa Bonita
4,Vanimo,PG,77.85,overcast clouds,-2.67,141.30,Vahmoneh Lodge Vanimo


In [3]:
# 2. Using the template add the city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Maximum Temperature</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 3a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in vacation_df.iterrows()]

# 3b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = vacation_df[["Lat", "Lng"]]

In [4]:
# 4a. Add a marker layer for each city to the map.

fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(marker_layer)
# 4b. Display the figure
fig


Figure(layout=FigureLayout(height='420px'))

In [5]:
# From the map above pick 4 cities and create a vacation itinerary route to travel between the four cities. 
# 5. Create DataFrames for each city by filtering the 'vacation_df' using the loc method. 
# Hint: The starting and ending city should be the same city.

#sort vacation df based on count of cities in a country
new_vacation_df = vacation_df[vacation_df['Country'].map(vacation_df["Country"].value_counts())>3].copy()
new_vacation_df = new_vacation_df.sort_values(by='Country')
vacation_start = new_vacation_df.iloc[0].copy()
vacation_end = new_vacation_df.iloc[0].copy()
vacation_stop1 = new_vacation_df.iloc[1].copy()
vacation_stop2 = new_vacation_df.iloc[2].copy()
vacation_stop3 = new_vacation_df.iloc[3].copy()

In [6]:
# 6. Get the latitude-longitude pairs as tuples from each city DataFrame using the to_numpy function and list indexing.
start= vacation_start[["Lat","Lng"]].to_numpy()
end = vacation_end[["Lat","Lng"]].to_numpy()
stop1 = vacation_stop1[["Lat","Lng"]].to_numpy()
stop2 = vacation_stop2[["Lat","Lng"]].to_numpy()
stop3 = vacation_stop3[["Lat","Lng"]].to_numpy()

In [8]:
# 7. Create a direction layer map using the start and end latitude-longitude pairs,
# and stop1, stop2, and stop3 as the waypoints. The travel_mode should be "DRIVING", "BICYCLING", or "WALKING".

travel_mode = input("How would you like to Travel(DRIVING, BICYCLING, or WALKING):")
start_end_combination_bic = gmaps.directions_layer(start,end, waypoints=[stop1, stop2, stop3],
        travel_mode=travel_mode)
fig = gmaps.figure(center=start, zoom_level=11)

fig.add_layer(start_end_combination_bic)
fig


Figure(layout=FigureLayout(height='420px'))

In [9]:
# 8. To create a marker layer map between the four cities.
#  Combine the four city DataFrames into one DataFrame using the concat() function.
itinerary_df = pd.concat([vacation_start, vacation_stop1, vacation_stop2, vacation_stop3],axis=1)
itinerary_df=itinerary_df.transpose()

In [10]:
# 9 Using the template add city name, the country code, the weather description and maximum temperature for the city. 
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in itinerary_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = itinerary_df[["Lat", "Lng"]]


In [11]:
# 11a. Add a marker layer for each city to the map.
fig = gmaps.figure(center=(-45.0, -45.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations,info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))